In [1]:
import pandas as pd

df = pd.read_csv('csv/food_rows.csv')

In [2]:
df.columns

Index(['ID', 'TYPE', 'PROGRAM', 'ORG PHONE', 'DAYS', 'HOURS', 'LONGITUDE',
       'LATITUDE', 'FULL ADDRESS', 'BOROUGH', 'ZICODE', 'WEEKS_OPEN', 'mon',
       'tue', 'wed', 'thur', 'fri', 'sat', 'sun'],
      dtype='object')

In [3]:
df['WEEKS_OPEN'].value_counts()

WEEKS_OPEN
2,4                29
1,3                26
1,2,3,4             9
2ND                 6
3RD                 5
4TH                 3
3                   3
4                   3
1                   3
2                   2
1ST                 2
2,3,4               2
4,2                 2
1,2,3               2
3,1                 2
CLOSED 5TH WED      1
1st                 1
,4                  1
4TH ONLY            1
1,2,4,5,3           1
1,3,2,4             1
1ST,3RD             1
BY APPT             1
SR ONLY             1
LAST                1
1,2,3,4,1,2,3,4     1
2,3                 1
1,3,5,2,4           1
1,2,3,4,4           1
2,LAST              1
Name: count, dtype: int64

In [4]:

# Function to standardize the week numbers, handling non-string inputs
def standardize_weeks(weeks_open):
    if isinstance(weeks_open, str):  # Only process if it's a string
        weeks_open = weeks_open.replace('1st', '1').replace('2ND', '2').replace('3RD', '3').replace('4TH', '4')
        weeks_open = weeks_open.replace('LAST', '4').replace('BY APPT', 'APPT').replace('SR ONLY', 'SR')
        weeks_open = weeks_open.replace('CLOSED 5TH WED', 'CLOSED').replace('4TH ONLY', '4')
        return weeks_open
    else:
        return weeks_open  # Return the value as-is if it's not a string

# Apply the standardization function to the WEEKS_OPEN column
df['WEEKS_OPEN_STANDARDIZED'] = df['WEEKS_OPEN'].apply(standardize_weeks)

In [5]:
df['WEEKS_OPEN_STANDARDIZED'].value_counts()

WEEKS_OPEN_STANDARDIZED
2,4                30
1,3                26
1,2,3,4             9
2                   8
3                   8
4                   7
1                   4
2,3,4               2
4,2                 2
3,1                 2
1ST                 2
1,2,3               2
1,2,3,4,4           1
SR                  1
CLOSED              1
2,3                 1
1,3,5,2,4           1
1,2,3,4,1,2,3,4     1
1,3,2,4             1
1ST,3               1
APPT                1
1,2,4,5,3           1
4 ONLY              1
,4                  1
Name: count, dtype: int64

In [6]:
def normalize_weeks(weeks_open):
    if isinstance(weeks_open, str):
        # Split the string into a list, remove duplicates, and sort the list
        weeks = sorted(set(week.strip() for week in weeks_open.split(',') if week.strip() != ''))
        return ','.join(weeks)
    else:
        return weeks_open  # Return the value as-is if it's not a string

df['WEEKS_OPEN_NORMALIZED'] = df['WEEKS_OPEN_STANDARDIZED'].apply(normalize_weeks)

In [7]:
df['WEEKS_OPEN_NORMALIZED'].value_counts()

WEEKS_OPEN_NORMALIZED
2,4          32
1,3          28
1,2,3,4      12
2             8
4             8
3             8
1             4
1ST           2
1,2,3,4,5     2
2,3,4         2
1,2,3         2
CLOSED        1
SR            1
2,3           1
1ST,3         1
APPT          1
4 ONLY        1
Name: count, dtype: int64

In [8]:
def handle_special_cases(weeks_open):
    if isinstance(weeks_open, str):
        # Replace special cases with their appropriate values
        weeks_open = weeks_open.replace('1ST', '1').replace('4 ONLY', '4')
        # Keep special cases as they are
        if weeks_open in ['CLOSED', 'SR', 'APPT']:
            return weeks_open
        else:
            # Normalize the weeks after replacing special cases
            weeks = sorted(set(week.strip() for week in weeks_open.split(',') if week.strip() != ''))
            return ','.join(weeks)
    else:
        return weeks_open  # Return the value as-is if it's not a string

df['WEEKS_OPEN_PROCESSED'] = df['WEEKS_OPEN_NORMALIZED'].apply(handle_special_cases)

In [9]:
df['WEEKS_OPEN_PROCESSED'].value_counts()

WEEKS_OPEN_PROCESSED
2,4          32
1,3          29
1,2,3,4      12
4             9
2             8
3             8
1             6
2,3,4         2
1,2,3         2
1,2,3,4,5     2
APPT          1
CLOSED        1
SR            1
2,3           1
Name: count, dtype: int64

In [10]:
# Drop rows where WEEKS_OPEN_PROCESSED is 'APPT', 'CLOSED', or 'SR'
df = df[~df['WEEKS_OPEN_PROCESSED'].isin(['APPT', 'CLOSED', 'SR'])]

# Display the DataFrame to confirm the rows were dropped
print(df[['WEEKS_OPEN_PROCESSED']].value_counts())

WEEKS_OPEN_PROCESSED
2,4                     32
1,3                     29
1,2,3,4                 12
4                        9
2                        8
3                        8
1                        6
1,2,3                    2
1,2,3,4,5                2
2,3,4                    2
2,3                      1
Name: count, dtype: int64


In [11]:
import numpy as np

# Step 1: Fill empty or NaN values with '1,2,3,4,5' to represent all weeks
df['WEEKS_OPEN_PROCESSED'] = df['WEEKS_OPEN_PROCESSED'].replace('', np.nan).fillna('1,2,3,4,5')

# Step 2: Create boolean columns for each week
df['WEEK_1'] = df['WEEKS_OPEN_PROCESSED'].apply(lambda x: '1' in x)
df['WEEK_2'] = df['WEEKS_OPEN_PROCESSED'].apply(lambda x: '2' in x)
df['WEEK_3'] = df['WEEKS_OPEN_PROCESSED'].apply(lambda x: '3' in x)
df['WEEK_4'] = df['WEEKS_OPEN_PROCESSED'].apply(lambda x: '4' in x)
df['WEEK_5'] = df['WEEKS_OPEN_PROCESSED'].apply(lambda x: '5' in x)

In [12]:
df.dtypes

ID                           int64
TYPE                        object
PROGRAM                     object
ORG PHONE                   object
DAYS                        object
HOURS                       object
LONGITUDE                  float64
LATITUDE                   float64
FULL ADDRESS                object
BOROUGH                     object
ZICODE                       int64
WEEKS_OPEN                  object
mon                           bool
tue                           bool
wed                           bool
thur                          bool
fri                           bool
sat                           bool
sun                           bool
WEEKS_OPEN_STANDARDIZED     object
WEEKS_OPEN_NORMALIZED       object
WEEKS_OPEN_PROCESSED        object
WEEK_1                        bool
WEEK_2                        bool
WEEK_3                        bool
WEEK_4                        bool
WEEK_5                        bool
dtype: object

In [13]:
df.drop(columns=['WEEKS_OPEN_STANDARDIZED', 'WEEKS_OPEN_NORMALIZED', 'WEEKS_OPEN'], inplace=True)

In [14]:
df.rename(columns={'WEEKS_OPEN_PROCESSED': 'WEEKS_OPEN'}, inplace=True)


In [15]:
# Display the first few rows to confirm the changes
print(df.head())

# Display the column names to ensure the changes were applied
print(df.columns)

      ID TYPE                                    PROGRAM       ORG PHONE  \
0  80015   FP                 THE BIBLE CHURCH OF CHRIST    718-293-1928   
1  80026   FP         BRONX SEVENTH DAY ADVENTIST CHURCH    646-353-8926   
2  80029   FP  BRONX TEMPLE SEVENTH DAY ADVENTIST CHURCH    718-842-4504   
3  80065   FP                  GETHSEMANE BAPTIST CHURCH    347-948-0772   
4  80081   FP                HOLY TABERNACLE CHURCH INC.  (718) 293-9862   

       DAYS           HOURS  LONGITUDE   LATITUDE  \
0       SUN           4-6PM -73.968168  40.693683   
1      THUR           2-5PM -73.955831  40.654492   
2  M,W,THUR  11:30AM-2:30PM -73.912934  40.668113   
3      THUR          8-10AM -73.963443  40.576623   
4       TUE        4:30-5PM -73.912934  40.668113   

                                        FULL ADDRESS   BOROUGH  ...   thur  \
0  138, Waverly Avenue, Clinton Hill, Brooklyn, K...  Brooklyn  ...  False   
1  Bethanie Eglise Adventiste du 7eme Jour, 2059-...  Brooklyn  ... 

In [16]:
check = df[['WEEKS_OPEN', 'WEEK_1', 'WEEK_2', 'WEEK_3', 'WEEK_4', 'WEEK_5']]

In [17]:
pd.set_option('display.max_rows', None)
check

,WEEKS_OPEN,WEEK_1,WEEK_2,WEEK_3,WEEK_4,WEEK_5
0,"2,4",False,True,False,True,False
1,"1,2,3,4,5",True,True,True,True,True
2,"1,2,3,4,5",True,True,True,True,True
3,"1,2,3,4,5",True,True,True,True,True
4,"1,2,3,4,5",True,True,True,True,True
5,"1,2,3,4,5",True,True,True,True,True
6,"1,2,3,4,5",True,True,True,True,True
7,"1,2,3,4,5",True,True,True,True,True
8,"1,2,3,4,5",True,True,True,True,True
9,"1,2,3,4,5",True,True,True,True,True


In [18]:
# Select the relevant columns for the database
df_to_insert = df[['ID', 'WEEKS_OPEN', 'WEEK_1', 'WEEK_2', 'WEEK_3', 'WEEK_4', 'WEEK_5']]

In [19]:
import os
from supabase import create_client, Client
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve the environment variables
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")

# Initialize Supabase client
supabase: Client = create_client(url, key)

In [22]:
# Rename the columns to lowercase
df.rename(columns={
    'WEEKS_OPEN': 'weeks_open',
    'WEEK_1': 'week_1',
    'WEEK_2': 'week_2',
    'WEEK_3': 'week_3',
    'WEEK_4': 'week_4',
    'WEEK_5': 'week_5'
}, inplace=True)

In [27]:
df_to_insert = df[['ID', 'weeks_open', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5']]
data_list = df_to_insert.to_dict(orient='records')

# Step 3: Upsert the data
response = supabase.table('food').upsert(data_list).execute()

# Step 4: Verify the data
verify_response = supabase.table('food').select('ID', 'weeks_open', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5').limit(5).execute()

print(verify_response.data)

[{'ID': 80345, 'weeks_open': '1,2,3,4,5', 'week_1': True, 'week_2': True, 'week_3': True, 'week_4': True, 'week_5': True}, {'ID': 81595, 'weeks_open': '1,2,3,4,5', 'week_1': True, 'week_2': True, 'week_3': True, 'week_4': True, 'week_5': True}, {'ID': 82033, 'weeks_open': '1,2,3,4,5', 'week_1': True, 'week_2': True, 'week_3': True, 'week_4': True, 'week_5': True}, {'ID': 83082, 'weeks_open': '1,2,3,4,5', 'week_1': True, 'week_2': True, 'week_3': True, 'week_4': True, 'week_5': True}, {'ID': 83311, 'weeks_open': '1,2,3,4,5', 'week_1': True, 'week_2': True, 'week_3': True, 'week_4': True, 'week_5': True}]
